In [1]:
#Notations : p prime, MesF Mestre test function, fnu inertia degree, lam = lambda

#MesF : Mestre test function. For these algorithms to be correct, MesF must be 
# Complactly supported in [-1,1], even function, positive Fourier transform

Mes(x) = (1-x)*cos(pi*x)+sin(pi*x)/pi


def liste_premier_lambda(lam):                         # Input : lam 
    C = floor(exp(lam))                                # Output : List of primes <= exp(lam)
    return prime_range(C+1)


#Only over Q
def Mestre_somme_premier_lambda(p,lam,MesF):            #Local Mestre sum          
    S,m,x = 0,1,p                                       
    while x<= exp(lam):
        S += ((floor(2*sqrt(x)) * x^(-1) * MesF(m*math.log(p)/lam.numerical_approx())).abs()).numerical_approx()
        x *= x
        m += 1
    return numerical_approx(log(p)*S)


   
#Only works over Q.    
def Mestre_somme_lambda(lam,MesF):                        # Global Mestre sum - this term is positive (and we substract it)
    Liste = liste_premier_lambda(lam)
    Somme = 0
    for p in Liste:
        Somme += (Mestre_somme_premier_lambda(p,lam,MesF)).numerical_approx()
    return 2*Somme                                        #In the final formula this sum must be multiplied by two

#Only work over Q                                           
def somme_mult_badprimes_lambda(p,lam,MesF):                 # Local sum in the case of p|N i.e multiplicative reduction      
    S = p^(-1)*MesF(math.log(p)/lam.numerical_approx())
    m = 2
    x = p^2
    while x <= exp(lam):
        S = ( S +   (   x^(-1)*MesF(m*math.log(p)/lam.numerical_approx()) ).abs()  ).numerical_approx()
        x = p * x
        m = m+1
    return numerical_approx(log(p)*S)

                                                              #Additive reduciton : the sum is 0 we don't code it


#Only over Q - uses the Wil bound
#input : lambda, MesF,two lists of bad primes : one for multiplicative red and one for additive red



def Mestre_somme_withbadprimes_lambda(lam,MesF,mult_badprimes,add_badprimes): #Global sum with mult and add reductions
    Liste = liste_premier_lambda(lam)                                              
    Somme = 0
    for pri in Liste:                                                  
        if pri in mult_badprimes :                               #Correct sum for multiplicative red
            Somme += somme_mult_badprimes_lambda(pri,lam,MesF)
        elif pri in add_badprimes: 
            Somme += 0                                      #Do nothing for additive reduction
        else:
            Somme += Mestre_somme_premier_lambda(pri,lam,MesF)   # In case of good reduction
    return 2*Somme                                             #factor 2 in the final sum - present in Mestre explicit formula                                                           




#remark : the list we give in input are lists of primes CONTAINED in the set of primes of bad reduction.
#The algorithm stays worect if we forgot some primes of bad reduction, but the result is lee precise 
# that is to say the final bound is lwoer that it would be



#Function defining the integrals terms appearing. Not correct if MesF is not completly supported in [-1,1]


def Value_Phi_0(MesF):      #The integral term which will be multiplied by the rank. Fourier transform of MesF eval at 0
    return 2*numerical_integral(MesF(x),0,1)[0]


Constant_Mlambda = 2*math.log(2*pi)     #constant appearing in the formula, let's stock it



def Mlambda2(lam,MesF):
    aux(x) = MesF(x) * exp(-lam*x)/(1-exp(-lam*x)) - MesF(0) * exp(-lam*x)/(lam*x)
    a = numerical_integral(aux(x),0,1)[0]
    b = numerical_integral(exp(-x)/x, lam.numerical_approx() , Infinity )[0] #depends on lam, cannot be stocked :(
    return 2*(lam*a - MesF(0)*b) +   MesF(0)*Constant_Mlambda       #still a foctor two at the end, like the sum




#Final lower bound 

#Only over Q
def minorant_conducteur_lambda(lam,MesF,rk):      #Without knowing anything on the conductor
    K = lam * rk * Value_Phi_0(MesF)+Mlambda2(lam,MesF)-Mestre_somme_lambda(lam,MesF)
    return exp(K)

#Only over Q
def minorant_conducteur_lambda_withbadprimes(lam,MesF,rk,mult_badprimes,add_badprimes): #knowing some primes of add and mul reduction
    K = lam * rk *Value_Phi_0(MesF) + Mlambda2(lam,MesF)-Mestre_somme_withbadprimes_lambda(lam,MesF,mult_badprimes,add_badprimes)
    return exp(K)



In [2]:
#In number fields, without knowing anything about the bad primes, using only the Weil bound

def somme_premier_degre_ramifi(p,lam,MesF,fnu):
    q = p^fnu
    S,m,x = 0,1,q
    while x<= exp(lam):
        S += (floor(2*sqrt(x)) * x^(-1) * MesF(m*math.log(q)/lam.numerical_approx())).numerical_approx()
        x *=  q
        m +=  1
    return numerical_approx(log(q)*S)



def Mestre_somme_lambda_nbfield(lam,MesF,K):        #lam = lambda, MesF mestre test function, K number field            
    Liste = liste_premier_lambda(lam)
    Somme = 0
    for p in Liste:
        for pp in K.primes_above(p):
            fnu = pp.residue_class_degree()
            Somme += (somme_premier_degre_ramifi(p,lam,MesF,fnu)).numerical_approx()
    return 2 * Somme             # un facteur de 2 car deux termes conjugués dans la somme de Mestre



def minorant_conducteur_lambda_nbfield(lam,MesF,rk,K):     # K nb field (chose not to give polynomial in case of tower extension)  
    n = K.absolute_degree()
    disc = K.absolute_discriminant()
    lowbound = lam * rk * Value_Phi_0(MesF) + n * Mlambda2(lam,MesF) - Mestre_somme_lambda_nbfield(lam,MesF,K)
    return exp(lowbound)/(disc^2)




#With Frobenius. Here the functions are exact, no need to take abd in the formulas.

def Somme_Frob(p,fnu,lam,MesF,Frob):
    q = p^fnu 
    delt = Frob^2-4*q
    alpha = (Frob-I*sqrt(-delt))/2
    beta = (Frob+I*sqrt(-delt))/2
    a = alpha
    b = beta
    Somme = 0
    m = 1
    x = q
    while x <= exp(lam):
        Somme += (MesF(m*math.log(q)/numerical_approx(lam))*(a+b)/x).numerical_approx()   # compute the sum
        a     *= alpha                                             # powers of roots of Frob increment
        b     *= beta
        x     *= q                                                 # denominator of the sum increment
        m     += 1
    finalSomme = 2 * Somme * math.log(q)                           # factor  2 in the Mestre formula
    return numerical_approx(finalSomme)


def Mini_Somme_Frob(p,fnu,lam,MesF):       #take the minimal over all possible fors, return the frob and the sum 
    bound = floor(2*sqrt(p^fnu))
    MinFrob = - bound
    MinSomme = Somme_Frob(p,fnu,lam,MesF,MinFrob)
    for Frobenius in range(-bound+1,bound+1):
        C = Somme_Frob(p,fnu,lam,MesF,Frobenius)
        if C < MinSomme:
            MinFrob = Frobenius
            MinSomme = C
    return (MinFrob,MinSomme)


def Mestre_Somme_Frob(lam,MesF,K):
    Listeprime = liste_premier_lambda(lam)
    BigSomme = 0
    ListeFrob = []
    for p in Listeprime:
        for prims in K.primes_above(p):
            fnu = prims.residue_class_degree()
            frobenius,value = Mini_Somme_Frob(p,fnu,lam,MesF)
            BigSomme += value.numerical_approx()
            ListeFrob += [(p,fnu,frobenius)]
    return (ListeFrob,BigSomme)

def minorant_conducteur_nbfield_Frobenius(lam,MesF,rk,K):   # K nb field  
    n = K.absolute_degree()
    discri = K.absolute_discriminant()
    Listefrobs, MestreSomme = Mestre_Somme_Frob(lam,MesF,K)
    lowbound = lam * rk * Value_Phi_0(MesF) + n * Mlambda2(lam,MesF) + MestreSomme
    return Listefrobs , exp(lowbound)/(discri^2)





def Somme_liste_Frob(lam,MesF,lisfrobs):   #Input: the list of primes considered, with their fnu and attributed frobenius
    S = 0
    for k in lisfrobs:
        p,fnu,Frob = k
        S += (Somme_Frob(p,fnu,lam,MesF,Frob)).numerical_approx()
    return S

def minorant_liste_Frob(lam,MesF,rk,K,lisfrobs):
    n = K.absolute_degree()
    discri = K.absolute_discriminant()
    Mestresomme = Somme_liste_Frob(lam,MesF,lisfrobs)
    lowbound = lam * rk * Value_Phi_0(MesF) + n * Mlambda2(lam,MesF) + Mestresomme  #here we add the sum (it's a negative term)
    return  exp(lowbound)/(discri^2)
   
    
    
    
    
    
    

    
    
def somme_split_mult_badprimes_lambda(p,lam,MesF):           # Local sum in the case of p|N i.e split multiplicative reduction      
    S = - p^(-1)*MesF(math.log(p)/lam.numerical_approx())
    m = 2
    x = p^2
    while x <= exp(lam):
        S = ( S +   (  (-1)^m* x^(-1)*MesF(m*math.log(p)/lam.numerical_approx()) ).abs()  ).numerical_approx()
        x = p * x
        m = m+1
    return numerical_approx(2*log(p)*S)  #factor 2 in the m=Mestre formula  

    
    
    
def Mestre_somme_withbadprimes_listfrobs(lam,MesF,lisfrobs,mult_badprimes,add_badprimes):#Global sum with mult and add reductions
    Somme = 0
    for k in lisfrobs:
        pri,fnu,Frob = k    #fnu = 1 systemtically                                              
        if pri in mult_badprimes :                                    #Correct sum for multiplicative red
            Somme += somme_split_mult_badprimes_lambda(pri,lam,MesF)    #negative term to minimize in this case
        elif pri in add_badprimes: 
            Somme += 0                                      #Do nothing for additive reduction
        else:
            Somme += (Somme_Frob(pri,1,lam,MesF,Frob)).numerical_approx()   # In case of good reduction
    return Somme         #the factor 2 has already been put locally                                                                                                



def minorant_conducteur_withbadprimes_listfrobs(lam,MesF,rk,lisfrobs,mult_badprimes,add_badprimes): #knowing some primes of add and mul reduction
    K = lam * rk *Value_Phi_0(MesF) + Mlambda2(lam,MesF) + Mestre_somme_withbadprimes_listfrobs(lam,MesF,lisfrobs,mult_badprimes,add_badprimes)
    return exp(K)



In [4]:
#   optimisation de la fonction de Mestre dans un sous espace de dimesion 5 


def fonctiontest(a0,a1,a2,a3,a4): #output convolution square of a0 + a1cos(2pix) + a2cos(4pix) + a3cos(6pix) +a4cos(pix)sur [-1/2,1/2]
    norm=sqrt(a0^2+1/2*(a1^2+a2^2+a3^2+a4^2) +2*(2/pi*a4*(a0+a1/3-a2/15+a3/35)))
    b0,b1,b2,b3,b4=a0/norm,a1/norm,a2/norm,a3/norm,a4/norm
    aux(x)=b0^2*g00(x)+b1^2*g11(x)+b2^2*g22(x)+b3^2*g33(x)+b4^2/2*Mes(x)+2*(b0*b1*g01(x)+b0*b2*g02(x)+b0*b3*g03(x)+b1*b2*g12(x)+b1*b3*g13(x)+b2*b3*g23(x)+b0*b4*gm0(x)+b1*b4*gm1(x)+b2*b4*gm2(x)+b3*b4*gm3(x))
    return aux

g00(x)= -x + 1
g01(x)= 1/2*sin(2*pi*x)/pi
g02(x)= -1/4*sin(4*pi*x)/pi
g03(x)= 1/6*sin(6*pi*x)/pi

g11(x)= 1/8*(2*pi*cos(2*pi*x) - sin(2*pi*x))/pi + 1/8*(2*(pi - 2*pi*x)*cos(2*pi*x) - sin(2*pi*x))/pi 
g12(x)= 1/3*sin(4*pi*x)/pi - 1/6*sin(2*pi*x)/pi
g13(x)= -3/16*sin(6*pi*x)/pi + 1/16*sin(2*pi*x)/pi

g22(x)= 1/16*(4*pi*cos(4*pi*x) - sin(4*pi*x))/pi + 1/16*(4*(pi - 2*pi*x)*cos(4*pi*x) - sin(4*pi*x))/pi
g23(x)= 3/10*sin(6*pi*x)/pi - 1/5*sin(4*pi*x)/pi

g33(x)= 1/24*(6*pi*cos(6*pi*x) - sin(6*pi*x))/pi + 1/24*(6*(pi - 2*pi*x)*cos(6*pi*x) - sin(6*pi*x))/pi

gm0(x) = cos(pi*x)/pi + 1/pi
gm1(x) = (1/6)*(sin(3/2*pi + 2*pi*x) + 3*sin(1/2*pi + 2*pi*x))/pi + 1/3*cos(pi*x)/pi
gm2(x) = (1/30)*(3*sin(5/2*pi + 4*pi*x) + 5*sin(3/2*pi + 4*pi*x))/pi - (1/15)*cos(pi*x)/pi
gm3(x) = (1/70)*(5*sin(7/2*pi + 6*pi*x) + 7*sin(5/2*pi + 6*pi*x))/pi + (1/35)*cos(pi*x)/pi

a00=1.27323954473516/2
a11=0.424413181578388
a22=-0.0848826363156775
a33=0.0363782727067189
b00=a00/sqrt(a00^2+1/2*(a11^2+a22^2+a33^2))
b11=a11/sqrt(a00^2+1/2*(a11^2+a22^2+a33^2))
b22=a22/sqrt(a00^2+1/2*(a11^2+a22^2+a33^2))
b33=a33/sqrt(a00^2+1/2*(a11^2+a22^2+a33^2))

def optimisitionfonction(lam,rk,K,lisfrobs,a0,a1,a2,a3,m0,pas,nb_iteration): #important condition : a0²+1/2(a1²+a2²+a3²+m0² 
    discri = K.absolute_discriminant()  #Notice I need to know the frobeniuses         + 2/pi(a0b0+a1b0/3 -a2b0/15 +a3b0/35)
    C00 = math.log(discri^2*minorant_liste_Frob(lam,g00,rk,K,lisfrobs))     #coefficient de ma fonction en a0,a1,a2,a3
    C01 = math.log(discri^2*minorant_liste_Frob(lam,g01,rk,K,lisfrobs)) 
    C02 = math.log(discri^2*minorant_liste_Frob(lam,g02,rk,K,lisfrobs)) 
    C03 = math.log(discri^2*minorant_liste_Frob(lam,g03,rk,K,lisfrobs)) 
    C11 = math.log(discri^2*minorant_liste_Frob(lam,g11,rk,K,lisfrobs)) 
    C12 = math.log(discri^2*minorant_liste_Frob(lam,g12,rk,K,lisfrobs)) 
    C13 = math.log(discri^2*minorant_liste_Frob(lam,g13,rk,K,lisfrobs)) 
    C22 = math.log(discri^2*minorant_liste_Frob(lam,g22,rk,K,lisfrobs)) 
    C23 = math.log(discri^2*minorant_liste_Frob(lam,g23,rk,K,lisfrobs)) 
    C33 = math.log(discri^2*minorant_liste_Frob(lam,g33,rk,K,lisfrobs))
    GM0 = math.log(discri^2*minorant_liste_Frob(lam,gm0,rk,K,lisfrobs))
    GM1 = math.log(discri^2*minorant_liste_Frob(lam,gm1,rk,K,lisfrobs))
    GM2 = math.log(discri^2*minorant_liste_Frob(lam,gm2,rk,K,lisfrobs))
    GM3 = math.log(discri^2*minorant_liste_Frob(lam,gm3,rk,K,lisfrobs))
    GG  = math.log(discri^2*minorant_liste_Frob(lam,Mes,rk,K,lisfrobs))
    b0 = a0
    b1 = a1
    b2 = a2
    b3 = a3
    b4 = m0
    for i in range(nb_iteration):
        tempvalue = (b0^2*C00+b1^2*C11+b2^2*C22+b3^2*C33+b4^2/2*GG+2*(b0*b1*C01+b0*b2*C02+b0*b3*C03+b1*b2*C12+b1*b3*C13+b2*b3*C23+b0*b4*GM0 +b1*b4*GM1 +b2*b4*GM2 +b3*b4*GM3)).numerical_approx() 
        gradient0 = ((2*b0*C00+2*b1*C01+2*b2*C02+2*b3*C03+2*b4*GM0) -tempvalue*(2*b0 + 2/pi*b4)).numerical_approx()    #gradient of the function
        gradient1 = ((2*b0*C01+2*b1*C11+2*b2*C12+2*b3*C13+2*b4*GM1)  -tempvalue*(b1  + 2/(3*pi)*b4)).numerical_approx()      
        gradient2 = ((2*b0*C02+2*b1*C12+2*b2*C22+2*b3*C23+2*b4*GM2)  -tempvalue*(b2  - 2/(15*pi)*b4)).numerical_approx()     
        gradient3 = ((2*b0*C03+2*b1*C13+2*b2*C23+2*b3*C33+2*b4*GM3)  -tempvalue*(b3  + 2/(35*pi)*b4)).numerical_approx() 
        gradient4 = ((2*b4*GG+2*b0*GM0+2*b1*GM1+2*b2*GM2+2*b3*GM3)  -tempvalue*(2*b4 + 2/pi*b0 + 2/(3*pi)*b1 - 2/(15*pi)*b2 +2/(35*pi)*b3)).numerical_approx() 
        normgrad = sqrt(gradient0^2+gradient1^2+gradient2^2+gradient3^2+gradient4^2)
        c0=(b0+gradient0*pas/normgrad).numerical_approx()                               # We move the original coordinates in the direction of the gradient 
        c1=(b1+gradient1*pas/normgrad).numerical_approx()                                             
        c2=(b2+gradient2*pas/normgrad).numerical_approx() 
        c3=(b3+gradient3*pas/normgrad).numerical_approx() 
        c4=(b4+gradient4*pas/normgrad).numerical_approx() 
        norm=sqrt(c0^2+1/2*(c1^2+c2^2+c3^2+c4^2) +2*(2/pi*c4*(c0+c1/3-c2/15+c3/35))) #final renormalisation to have F(0)=1                                      
        b0=(c0/norm).numerical_approx()
        b1=(c1/norm).numerical_approx()
        b2=(c2/norm).numerical_approx()
        b3=(c3/norm).numerical_approx()
        b4=(c4/norm).numerical_approx()
    finavalue=b0^2*C00+b1^2*C11+b2^2*C22+b3^2*C33+b4^2/2*GG+2*(b0*b1*C01 + b0*b2*C02 + b0*b3*C03 + b1*b2*C12 + b1*b3*C13 + b2*b3*C23 + b0*b4*GM0 + b1*b4*GM1 + b2*b4*GM2 + b3*b4*GM3 )
    return (b0,b1,b2,b3,b4,(1/discri^2)*exp(finavalue))



def optimisitionfonction_badred(lam,rk,K,lisfrobs,mult_badprimes,add_badprimes,a0,a1,a2,a3,m0,pas,nb_iteration): #important condition : a0²+1/2(a1²+a2²+a3²+m0² 
    discri = K.absolute_discriminant()  #Notice I need to know the frobeniuses         + 2/pi(a0b0+a1b0/3 -a2b0/15 +a3b0/35)
    C00 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g00,rk,lisfrobs,mult_badprimes,add_badprimes))     #coefficient de ma fonction en a0,a1,a2,a3
    C01 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g01,rk,lisfrobs,mult_badprimes,add_badprimes)) 
    C02 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g02,rk,lisfrobs,mult_badprimes,add_badprimes)) 
    C03 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g03,rk,lisfrobs,mult_badprimes,add_badprimes)) 
    C11 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g11,rk,lisfrobs,mult_badprimes,add_badprimes))
    C12 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g12,rk,lisfrobs,mult_badprimes,add_badprimes))
    C13 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g13,rk,lisfrobs,mult_badprimes,add_badprimes))
    C22 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g22,rk,lisfrobs,mult_badprimes,add_badprimes))
    C23 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g23,rk,lisfrobs,mult_badprimes,add_badprimes))
    C33 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,g33,rk,lisfrobs,mult_badprimes,add_badprimes))
    GM0 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,gm0,rk,lisfrobs,mult_badprimes,add_badprimes))
    GM1 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,gm1,rk,lisfrobs,mult_badprimes,add_badprimes))
    GM2 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,gm2,rk,lisfrobs,mult_badprimes,add_badprimes))
    GM3 = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,gm3,rk,lisfrobs,mult_badprimes,add_badprimes))
    GG  = math.log(discri^2*minorant_conducteur_withbadprimes_listfrobs(lam,Mes,rk,lisfrobs,mult_badprimes,add_badprimes))
    b0 = a0
    b1 = a1
    b2 = a2
    b3 = a3
    b4 = m0
    for i in range(nb_iteration):
        tempvalue = (b0^2*C00+b1^2*C11+b2^2*C22+b3^2*C33+b4^2/2*GG+2*(b0*b1*C01+b0*b2*C02+b0*b3*C03+b1*b2*C12+b1*b3*C13+b2*b3*C23+b0*b4*GM0 +b1*b4*GM1 +b2*b4*GM2 +b3*b4*GM3)).numerical_approx() 
        gradient0 = ((2*b0*C00+2*b1*C01+2*b2*C02+2*b3*C03+2*b4*GM0) -tempvalue*(2*b0 + 2/pi*b4)).numerical_approx()    #gradient of the function
        gradient1 = ((2*b0*C01+2*b1*C11+2*b2*C12+2*b3*C13+2*b4*GM1)  -tempvalue*(b1  + 2/(3*pi)*b4)).numerical_approx()      
        gradient2 = ((2*b0*C02+2*b1*C12+2*b2*C22+2*b3*C23+2*b4*GM2)  -tempvalue*(b2  - 2/(15*pi)*b4)).numerical_approx()     
        gradient3 = ((2*b0*C03+2*b1*C13+2*b2*C23+2*b3*C33+2*b4*GM3)  -tempvalue*(b3  + 2/(35*pi)*b4)).numerical_approx() 
        gradient4 = ((2*b4*GG+2*b0*GM0+2*b1*GM1+2*b2*GM2+2*b3*GM3)  -tempvalue*(2*b4 + 2/pi*b0 + 2/(3*pi)*b1 - 2/(15*pi)*b2 +2/(35*pi)*b3)).numerical_approx() 
        normgrad = sqrt(gradient0^2+gradient1^2+gradient2^2+gradient3^2+gradient4^2)
        c0=(b0+gradient0*pas/normgrad).numerical_approx()                               # We move the original coordinates in the direction of the gradient 
        c1=(b1+gradient1*pas/normgrad).numerical_approx()                                             
        c2=(b2+gradient2*pas/normgrad).numerical_approx() 
        c3=(b3+gradient3*pas/normgrad).numerical_approx() 
        c4=(b4+gradient4*pas/normgrad).numerical_approx() 
        norm=sqrt(c0^2+1/2*(c1^2+c2^2+c3^2+c4^2) +2*(2/pi*c4*(c0+c1/3-c2/15+c3/35))) #final renormalisation to have F(0)=1                                      
        b0=(c0/norm).numerical_approx()
        b1=(c1/norm).numerical_approx()
        b2=(c2/norm).numerical_approx()
        b3=(c3/norm).numerical_approx()
        b4=(c4/norm).numerical_approx()
    finavalue=b0^2*C00+b1^2*C11+b2^2*C22+b3^2*C33+b4^2/2*GG+2*(b0*b1*C01 + b0*b2*C02 + b0*b3*C03 + b1*b2*C12 + b1*b3*C13 + b2*b3*C23 + b0*b4*GM0 + b1*b4*GM1 + b2*b4*GM2 + b3*b4*GM3 )
    return (b0,b1,b2,b3,b4,(1/discri^2)*exp(finavalue))


# To find the best lambda with 0.01 precision:
def optimi_lam1(MesF,rk,K):
    earlylam = 1
    earlyval = minorant_conducteur_lambda_nbfield(earlylam,MesF,rk,K) 
    while minorant_conducteur_lambda_nbfield(earlylam + 1,MesF,rk,K) > earlyval:
        earlyval = minorant_conducteur_lambda_nbfield(earlylam + 1,MesF,rk,K)
        earlylam = earlylam + 1
    midlam = earlylam-0.9
    midval = minorant_conducteur_lambda_nbfield(midlam,MesF,rk,K)
    while minorant_conducteur_lambda_nbfield(midlam + 0.1,MesF,rk,K) > midval:
        midval = minorant_conducteur_lambda_nbfield(midlam + 0.1,MesF,rk,K)
        midlam = midlam + 0.1
    finlam = midlam-0.09
    finval = minorant_conducteur_lambda_nbfield(finlam,MesF,rk,K)
    while minorant_conducteur_lambda_nbfield(finlam+0.01,MesF,rk,K) > finval:
        finval = minorant_conducteur_lambda_nbfield(finlam+0.01,MesF,rk,K)
        finlam += 0.01
    return (finlam)

def optimi_lam(MesF,rk,K):
    midlam = 1
    midval = minorant_conducteur_nbfield_Frobenius(midlam,MesF,rk,K)[1] 
    while minorant_conducteur_nbfield_Frobenius(midlam + 0.1,MesF,rk,K)[1] > midval:
        midval = minorant_conducteur_nbfield_Frobenius(midlam + 0.1,MesF,rk,K)[1]
        midlam = midlam + 0.1
    finlam = midlam-0.09
    finval = minorant_conducteur_nbfield_Frobenius(finlam,MesF,rk,K)[1]
    while minorant_conducteur_nbfield_Frobenius(finlam+0.01,MesF,rk,K)[1] > finval:
        finval = minorant_conducteur_nbfield_Frobenius(finlam+0.01,MesF,rk,K)[1]
        finlam += 0.01
    return (finlam)
        
    
    
def optimi_lam_badred(MesF,rk,mult_badprimes,add_badprimes):
    earlylam = 1
    earlyval = minorant_conducteur_lambda_withbadprimes(earlylam,MesF,rk,mult_badprimes,add_badprimes)
    while minorant_conducteur_lambda_withbadprimes(earlylam + 1,MesF,rk,mult_badprimes,add_badprimes)> earlyval:
        earlyval = minorant_conducteur_lambda_withbadprimes(earlylam + 1,MesF,rk,mult_badprimes,add_badprimes)
        earlylam = earlylam + 1
    midlam = earlylam-0.9
    midval = minorant_conducteur_lambda_withbadprimes(midlam,MesF,rk,mult_badprimes,add_badprimes) 
    while minorant_conducteur_lambda_withbadprimes(midlam+0.1,MesF,rk,mult_badprimes,add_badprimes) > midval:
        midval = minorant_conducteur_lambda_withbadprimes(midlam+0.1,MesF,rk,mult_badprimes,add_badprimes) 
        midlam = midlam + 0.1
    finlam = midlam-0.09
    finval = minorant_conducteur_lambda_withbadprimes(finlam,MesF,rk,mult_badprimes,add_badprimes)
    while minorant_conducteur_lambda_withbadprimes(finlam+0.01,MesF,rk,mult_badprimes,add_badprimes) > finval:
        finval = minorant_conducteur_lambda_withbadprimes(finlam+0.01,MesF,rk,mult_badprimes,add_badprimes)
        finlam += 0.01
    return (finlam)

In [5]:
def tableaucomparatif(rang,liste_lam,dict_fonction):
    tableau=[]
    for k in dict_fonction.keys():
        F=dict_fonction[k]
        lambda_opti=0
        minorant_opti=0
        for lam in liste_lam:
            tyme = cputime()
            m = minorant_conducteur_lambda_withbadprimes(lam,F,rang,[],[])
            comptime=str(cputime(tyme))[:8]
            if  m > minorant_opti:
                lambda_opti = lam
                minorant_opti = m
        tableau=tableau+[(rang,k,lambda_opti.n(8),minorant_opti,comptime,"TBA7","TBA8","TBA9")]   
    return latex(table(tableau, header_row=["rank", "test function","lambda","Mestre bound","computation time","real bound","prooved","source"], frame=True))

def tableaucourbeminimale(liste_lam,dict_fonction):
    tableau=[]
    for rang in range(5):
        for k in dict_fonction.keys():
            lambda_opti=0
            minorant_opti=0
            F=dict_fonction[k]
            tyme = cputime()
            for lam in liste_lam:
                m = minorant_conducteur_lambda_withbadprimes(lam,F,rang,[],[])
                comptime=str(cputime(tyme))[:8]
                if  m > minorant_opti:
                    lambda_opti = lam
                    minorant_opti = m
            tableau=tableau+[(rang,k,lambda_opti.n(8),minorant_opti,comptime,"TBA1","TBA2","TBA3")]   
    return latex(table(tableau, header_row=["Fonction test","rang","minorant de Mestre","minorant réel","prouvé","source"], frame=True))

def tableaucomparatif_mauvaispremiers(rang,liste_lam,dict_fonction,mauvaispremiers1,mauvaispremiers2):
    tableau=[]
    for k in dict_fonction.keys():
        F=dict_fonction[k]
        lambda_opti=0
        minorant_opti=0
        for lam in liste_lam:
            tyme = cputime()
            m = minorant_conducteur_lambda_withbadprimes(lam,F,rang,mauvaispremiers1,mauvaispremiers2)
            comptime=str(cputime(tyme))[:8]
            if  m > minorant_opti:
                lambda_opti = lam
                minorant_opti = m
        tableau=tableau+[((rang,mauvaispremiers1,mauvaispremiers2),k,lambda_opti.n(12),round(minorant_opti,ndigits=3),comptime,"TBA4","TBA5","TBA6")]   
    return latex(table(tableau, header_row=["rank and reduction", "test function","lambda","Mestre bound","computation time","real bound","prooved","source"], frame=True))

def tableaucourbeminimale_mauvaispremiers(liste_lam,dict_fonction,mauvaispremiers1,mauvaispremiers2):
    tableau=[]
    for rank in range(5):
        for k in dict_fonction.keys():
            lambda_opti=0
            minorant_opti=0
            F=dict_fonction[k]
            tyme = cputime()
            for lam in liste_lam:
                m = minorant_conducteur_lambda_withbadprimes(lam,F,rank,mauvaispremiers1,mauvaispremiers2)
                comptime=str(cputime(tyme))[:8]
                if  m > minorant_opti:
                    lambda_opti = lam
                    minorant_opti = m
            tableau=tableau+[((rank,mauvaispremiers1,mauvaispremiers2),k,lambda_opti.n(8),round(minorant_opti,ndigits=3),comptime,"TBAx","TBAy","TBAz")]   
    return latex(table(tableau, header_row=["rank","test func","lambda","Mestre bound","realbound","prooved?","source"], frame=True))


In [6]:
# Les exemples de fonctions de Mestre


C(x)=-x^5 + 5*x^3 - 5*x^2 + 1         #carré de convolution de 1-(2x)^2. Ressemble beaucoup à celle de Mestre.

J(x)=1/2*pi*((2*pi - 4*pi*x + (pi - 2*pi*x)*cos(pi*x) - #carré de convolution de la fonction 1+cos(2*pi*x) sur [-1/2,1/2].
4*sin(-1/2*pi + pi*x) + sin(pi*x) + 4)/pi + (2*pi + (pi + 4)*cos(pi*x) + sin(pi*x) + 4)/pi)/(3*pi + 8)


K=piecewise([ ([0,1/2],6*x^3 - 3*x + 1) , (RealSet.open_closed(1/2,1),-2*x^3 + 3*x - 1)    ])
#carré de vonvolution de la fonction x. Une bonne idée, on a une nouvelle forme de fonction qui marche bien - cf graphe below




#La liste des lambda utilisés dans les comparaisons
listelam1 = [i/2 for i in range(1,11)]
listelam2 = [i/100 for i in range(100,200)]
listelam3 = [i/100 for i in range(200,250)]
listelam4 = [i/100 for i in range(200,300)]

#La liste des fonctions comparées. J'ai déjà essayé avec plus d'une fonction ca marche bien
L_fonctions1 = [Mes,C,J]
L_fonctions2=[Mes,K]

dictfunction={}
dictfunction["Mes(x)"]= Mes

#dictfunction["C(x)"]= C
#dictfunction["K(x)"]= K
#dictfunction["J(x)"]= J
#Lfonctions=[M,C,K,J]

In [10]:
rank = 0
listelam2 = [i/100 for i in range(100,200)]
dictfunction={}
dictfunction["Mes(x)"]= Mes

tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[])


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(0, \left[\right], \left[\right]\right)$ & Mes(x) & $1.3$ & $10.3935616135172$ & 0.007581 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

In [23]:
for prime in [2,3,5,7]:
    print()
    print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[prime],[]))
    print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[prime]))
    print()




\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(0, \left[2\right], \left[\right]\right)$ & Mes(x) & $1.47$ & $12.956$ & 0.007175 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}
\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(0, \left[\right], \left[2\right]\right)$ & Mes(x) & $1.68$ & $17.293$ & 0.011744 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(0, \left[3\right], \left[\right]\right)$ & Mes(x) & $1.49$ & $10.823$ & 0.007690 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}
\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mes

In [32]:
rank = 0
listelam2 = [i/100 for i in range(100,200)]
dictfunction={}
dictfunction["Mes(x)"]= Mes



print()
print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[7],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[3]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[7]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2,3],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2,5],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3,5],[]))
print()


print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[3]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[3]))
print()



\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(0, \left[2\right], \left[\right]\right)$ & Mes(x) & $1.47$ & $12.956$ & 0.011820 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(0, \left[3\right], \left[\right]\right)$ & Mes(x) & $1.49$ & $10.823$ & 0.007873 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(0, \left[5\right], \left[\right]\right)$ & Mes(x) & $1.31$ & $10.394$ & 0.008945 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Me

In [33]:
rank = 1
listelam2 = [i/100 for i in range(100,200)]
dictfunction={}
dictfunction["Mes(x)"]= Mes



print()
print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[7],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[3]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[7]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2,3],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2,5],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3,5],[]))
print()


print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[3]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[3]))
print()



\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[2\right], \left[\right]\right)$ & Mes(x) & $1.96$ & $51.571$ & 0.007215 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[3\right], \left[\right]\right)$ & Mes(x) & $1.99$ & $44.926$ & 0.006920 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[5\right], \left[\right]\right)$ & Mes(x) & $1.69$ & $34.588$ & 0.007134 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Me

In [35]:
rank = 1
listelam2 = [i/100 for i in range(200,300)]
dictfunction={}
dictfunction["Mes(x)"]= Mes



print()
print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[7],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[3]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[],[7]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2,3],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2,5],[]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3,5],[]))
print()


print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[3]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[2],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[3],[5]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[2]))
print()

print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5],[3]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[2\right], \left[\right]\right)$ & Mes(x) & $2.00$ & $51.503$ & 0.012486 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[3\right], \left[\right]\right)$ & Mes(x) & $2.00$ & $44.929$ & 0.012238 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[5\right], \left[\right]\right)$ & Mes(x) & $2.00$ & $32.845$ & 0.012500 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Me

In [37]:
rank = 1
listelam2 = [i/100 for i in range(100,200)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[5,7],[]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[5, 7\right], \left[\right]\right)$ & Mes(x) & $1.69$ & $34.588$ & 0.007112 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [42]:
minorant_conducteur_lambda_withbadprimes(1.8,Mes,1,[5,7],[])

34.3298670760271

In [69]:
rank = 5
listelam2 = [i/10 for i in range(40,50)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam2,dictfunction,[19047851],[]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(5, \left[19047851\right], \left[\right]\right)$ & Mes(x) & $4.10$ & $312851.845$ & 0.037831 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [7]:
rank = 1
listelam2 = [i/100 for i in range(150,300)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(1,listelam2,dictfunction,[],[2,3]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(1, \left[\right], \left[2, 3\right]\right)$ & Mes(x) & $2.65$ & $189.709$ & 0.009795 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [9]:
rank = 2
listelam = [i/100 for i in range(150,300)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[],[]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(2, \left[\right], \left[\right]\right)$ & Mes(x) & $2.16$ & $161.888$ & 0.012712 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [11]:
rank = 2
listelam = [i/100 for i in range(300,400)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[2],[3]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(2, \left[2\right], \left[3\right]\right)$ & Mes(x) & $3.05$ & $983.802$ & 0.022453 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [13]:
rank = 3
listelam = [i/100 for i in range(250,300)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[],[]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(3, \left[\right], \left[\right]\right)$ & Mes(x) & $2.77$ & $1224.679$ & 0.012266 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [14]:
rank = 3
listelam = [i/100 for i in range(300,350)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[],[5]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(3, \left[\right], \left[5\right]\right)$ & Mes(x) & $3.28$ & $2381.154$ & 0.014694 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [17]:
rank = 7
listelam = [i/100 for i in range(500,550)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[],[]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(7, \left[\right], \left[\right]\right)$ & Mes(x) & $5.08$ & $535075397.809$ & 0.057089 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [18]:
rank = 7
listelam = [i/100 for i in range(500,550)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[13,17],[]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(7, \left[13, 17\right], \left[\right]\right)$ & Mes(x) & $5.42$ & $2178753233.183$ & 0.056634 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [19]:
rank = 2
listelam = [i/100 for i in range(300,400)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[2,5,11],[3]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(2, \left[2, 5, 11\right], \left[3\right]\right)$ & Mes(x) & $3.58$ & $2180.596$ & 0.020451 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}



In [22]:
rank = 7
listelam = [i/100 for i in range(500,550)]
dictfunction={}
dictfunction["Mes(x)"]= Mes
print()
print(tableaucomparatif_mauvaispremiers(rank,listelam,dictfunction,[3],[2]))
print()


\begin{tabular}{|l|l|l|l|l|l|l|l|} \hline
rank and reduction & test function & lambda & Mestre bound & computation time & real bound & prooved & source \\ \hline \hline
$\left(7, \left[3\right], \left[2\right]\right)$ & Mes(x) & $5.36$ & $45263532599.863$ & 0.056565 & TBA4 & TBA5 & TBA6 \\ \hline
\end{tabular}

